In [2]:
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 800 kB/s eta 0:00:01


In [3]:
!pip install joblib

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib
np.random.seed(2020)


def lgb_model_age(train_x, test_x,train_y, test_y):
    model = lgb.LGBMClassifier (objective = 'multiclass',
                                learning_rate=0.06787,
                                num_leaves =57,
                                colsample_bytree = 0.9712,
                                reg_alpha= 0.06883,
                                reg_lambda= 0.09217,
                                subsample =  0.9,
                                n_estimators = 10000)
    model.fit(train_x,train_y,early_stopping_rounds=100,eval_set=[(train_x,train_y),(test_x,test_y)],verbose = 10)
    
    
    # 模型存储
    joblib.dump(model, 'w2v_lgb_age_2_0.2.pkl')
    # 模型加载
    # model = joblib.load('lgb_cough.pkl')
    # # 模型预测
    # y_t_pred = model.predict(test_x, num_iteration=model.best_iteration_)
    
    y_t_pred = model.predict(test_x)

    # print(model.get_score(importance_type='weight'))
    cm = confusion_matrix(test_y, y_t_pred)
    np.set_printoptions(precision=3)                                    # 显示精度
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # 将样本矩阵转化为比率


    print('age************************************')
    print('confusion_matrix is \n {:} \n '.format(cm_normalized))
    print('test acc is \n {:} \n '.format(np.sum(test_y==y_t_pred)/len(test_y)))
    print(classification_report(test_y,y_t_pred))
    print('accuracy is %f , sen is %f,spe is %f ' % (accuracy_score(test_y, y_t_pred) * 100,
                                                     cm_normalized[0][0],cm_normalized[1][1] ))

    return accuracy_score(test_y, y_t_pred)

def lgb_model_gender(train_x, test_x,train_y, test_y):
    model = lgb.LGBMClassifier (objective = 'binary',
                               learning_rate=0.0376,
                                num_leaves = 22,
                                colsample_bytree = 0.23,
                                reg_alpha= 0.096,
                                reg_lambda= 0.0899,
                                subsample =  0.99,
                                n_estimators = 10000)
    model.fit(train_x,train_y,early_stopping_rounds=100,eval_set=[(train_x,train_y),(test_x,test_y)],verbose = 10)
    
    # 模型存储
    joblib.dump(model, 'w2v_lgb_gender_2_0.2.pkl')
    
    y_t_pred = model.predict(test_x)

    # print(model.get_score(importance_type='weight'))
    cm = confusion_matrix(test_y, y_t_pred)
    np.set_printoptions(precision=3)                                    # 显示精度
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # 将样本矩阵转化为比率

    print('gender************************************')
    print('confusion_matrix is \n {:} \n '.format(cm_normalized))
    print('test acc is \n {:} \n '.format(np.sum(test_y==y_t_pred)/len(test_y)))
    print(classification_report(test_y,y_t_pred))
    print('accuracy is %f , sen is %f,spe is %f ' % (accuracy_score(test_y, y_t_pred) * 100,
                                                     cm_normalized[0][0],cm_normalized[1][1] ))
    return accuracy_score(test_y, y_t_pred)
def lgb_model_gender2(train_x, test_x,train_y, test_y):
#     model = lgb.LGBMRegressor(n_estimators = 10000)
#     model.fit(train_x,train_y,early_stopping_rounds=100,eval_set=[(train_x,train_y),(test_x,test_y)],verbose = 10)
    
    # 模型存储
#     joblib.dump(model, 'w2v_gender_reg_2.pkl')
    model = joblib.load('w2v_gender_reg_2.pkl')
    
    y_t_pred = model.predict(test_x)
    
    y_t_pred = [2 if i >1.5 else 1 for i in y_t_pred]

    # print(model.get_score(importance_type='weight'))
    cm = confusion_matrix(test_y, y_t_pred)
    np.set_printoptions(precision=3)                                    # 显示精度
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # 将样本矩阵转化为比率

    print('gender************************************')
    print('confusion_matrix is \n {:} \n '.format(cm_normalized))
    print('test acc is \n {:} \n '.format(np.sum(test_y==y_t_pred)/len(test_y)))
    print(classification_report(test_y,y_t_pred))
    print('accuracy is %f , sen is %f,spe is %f ' % (accuracy_score(test_y, y_t_pred) * 100,
                                                     cm_normalized[0][0],cm_normalized[1][1] ))
    return accuracy_score(test_y, y_t_pred)

def load_data():
    # user
    data = pd.read_csv('../w2v_feat_data/train_data.csv')
    label = data[['age','gender']]
    
    data = data.drop(['user_id','age','gender'],axis = 1)
    return data,label

In [5]:
data,label = load_data()

In [6]:
#划分age的训练和测试数据
train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2,random_state=2020)

In [7]:
# stand = StandardScaler()
# stand.fit(train_age_x)
# train_age_x, test_age_x = stand.transform(train_age_x),stand.transform(test_age_x)
# 训练模型
acc_age = lgb_model_age(train_x, test_x,train_y['age'], test_y['age'])
# 训练模型
acc_gender = lgb_model_gender(train_x, test_x,train_y['gender'], test_y['gender'])

print(acc_age+acc_gender)

# acc_gender2 = lgb_model_gender2(train_x, test_x,train_y['gender'], test_y['gender'])

Training until validation scores don't improve for 100 rounds
[10]	training's multi_logloss: 1.91644	valid_1's multi_logloss: 1.92408
[20]	training's multi_logloss: 1.84782	valid_1's multi_logloss: 1.86121
[30]	training's multi_logloss: 1.79936	valid_1's multi_logloss: 1.81791
[40]	training's multi_logloss: 1.76242	valid_1's multi_logloss: 1.786
[50]	training's multi_logloss: 1.73287	valid_1's multi_logloss: 1.7614
[60]	training's multi_logloss: 1.70835	valid_1's multi_logloss: 1.74169
[70]	training's multi_logloss: 1.68736	valid_1's multi_logloss: 1.72545
[80]	training's multi_logloss: 1.66909	valid_1's multi_logloss: 1.71179
[90]	training's multi_logloss: 1.65291	valid_1's multi_logloss: 1.70017
[100]	training's multi_logloss: 1.63821	valid_1's multi_logloss: 1.69006
[110]	training's multi_logloss: 1.6248	valid_1's multi_logloss: 1.6812
[120]	training's multi_logloss: 1.61241	valid_1's multi_logloss: 1.67339
[130]	training's multi_logloss: 1.60093	valid_1's multi_logloss: 1.66655
[14